In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
import tensorflow_datasets as tfds

In [35]:
ds, ds_info = tfds.load('malaria', with_info=True,as_supervised=True,shuffle_files=True,split=['train'])


In [36]:
ds,ds_info

([<_PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>],
 tfds.core.DatasetInfo(
     name='malaria',
     full_name='malaria/1.0.0',
     description="""
     The Malaria dataset contains a total of 27,558 cell images with equal instances
     of parasitized and uninfected cells from the thin blood smear slide images of
     segmented cells.
     """,
     homepage='https://lhncbc.nlm.nih.gov/publication/pub9932',
     data_path='/root/tensorflow_datasets/malaria/1.0.0',
     file_format=tfrecord,
     download_size=337.08 MiB,
     dataset_size=317.62 MiB,
     features=FeaturesDict({
         'image': Image(shape=(None, None, 3), dtype=uint8),
         'label': ClassLabel(shape=(), dtype=int64, num_classes=2),
     }),
     supervised_keys=('image', 'label'),
     disable_shuffling=False,
     splits={
         'train': <SplitInfo num_examples=27558, num_shards=4>,
     },
     citation="""@ar

In [84]:
for image,label in ds[0].take(1):
  print(image)
  print('lebel',label)
  

tf.Tensor(
[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]], shape=(145, 148, 3), dtype=uint8)
lebel tf.Tensor(1, shape=(), dtype=int64)


In [117]:
def split_train_test_val(dataset,train_ratio,val_ratio):
  dataset_size=len(dataset)
  train_dataset=dataset.take(int(train_ratio*dataset_size))
  
  train_dataset_size=len(train_dataset)
  print(train_dataset_size)


  val_test_dataset=dataset.skip(int(train_ratio*dataset_size))
  val_dataset=val_test_dataset.take(int(val_ratio*dataset_size))
   
  val_dataset_size=len(val_dataset)
  print(val_dataset_size)


  test_dataset=val_test_dataset.skip(int(val_ratio*dataset_size))
   
  test_dataset_size=len(test_dataset)
  print(test_dataset_size)

  return train_dataset,val_dataset,test_dataset

In [118]:
train_dataset,val_dataset,test_dataset=split_train_test_val(ds[0],0.8,0.1)

22046
2755
2757


In [119]:
img_shape=224
def resizing(img,label):
  return tf.image.resize(img,(img_shape,img_shape))/255.0,label

 

In [120]:
train_dataset=train_dataset.map(resizing)

In [121]:
val_dataset=val_dataset.map(resizing)

In [122]:
test_dataset=test_dataset.map(resizing)

In [123]:
train_dataset,test_dataset,val_dataset

(<_MapDataset element_spec=(TensorSpec(shape=(224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>,
 <_MapDataset element_spec=(TensorSpec(shape=(224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>,
 <_MapDataset element_spec=(TensorSpec(shape=(224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>)

In [92]:
for image,label in test_dataset.take(1):
  print(image,label)

tf.Tensor(
[[[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 ...

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]], shape=(224, 224, 3), dtype=float32) tf.Tensor(0, shape=(), dtype=int64)


In [124]:
Batch_size=32
train_dataset=train_dataset.shuffle(buffer_size=8).batch(Batch_size).prefetch(tf.data.AUTOTUNE)


In [125]:
val_dataset=val_dataset.shuffle(buffer_size=8).batch(Batch_size).prefetch(tf.data.AUTOTUNE)

In [126]:
train_dataset,test_dataset,val_dataset

(<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>,
 <_MapDataset element_spec=(TensorSpec(shape=(224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>,
 <_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>)

In [97]:
model=tf.keras.Sequential()

In [98]:
model.add(tf.keras.layers.Conv2D(8,3,input_shape=(224,224,3)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(10,activation='tanh'))

In [99]:
model.add(tf.keras.layers.Dense(80,activation='relu'))

In [100]:
model.add(tf.keras.layers.BatchNormalization())

In [101]:
model.add(tf.keras.layers.Dense(810,activation='sigmoid'))

In [102]:
model.add(tf.keras.layers.Dense(10,activation='tanh'))

In [103]:
model.add(tf.keras.layers.Dense(1,activation='sigmoid'))

In [104]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 222, 222, 8)       224       
                                                                 
 flatten_2 (Flatten)         (None, 394272)            0         
                                                                 
 dense_10 (Dense)            (None, 10)                3942730   
                                                                 
 dense_11 (Dense)            (None, 80)                880       
                                                                 
 batch_normalization_2 (Batc  (None, 80)               320       
 hNormalization)                                                 
                                                                 
 dense_12 (Dense)            (None, 810)               65610     
                                                      

In [105]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics='accuracy')

In [106]:
model.fit(train_dataset,validation_data=val_dataset,epochs=2)

Epoch 1/2
689/689 [==============================] - 50s 69ms/step - loss: 0.6944 - accuracy: 0.4975 - val_loss: 0.6936 - val_accuracy: 0.4962
Epoch 2/2
689/689 [==============================] - 44s 64ms/step - loss: 0.6934 - accuracy: 0.4969 - val_loss: 0.6937 - val_accuracy: 0.4933


In [130]:
test_data=test_dataset.batch(1)

In [131]:
len(test_data)

2757

In [76]:
model.evaluate(test_data)[1]

1/1 [==============================] - 12s 12s/step - loss: 0.6921 - accuracy: 0.5312


In [80]:
parasite(x)

Negative


In [109]:
model.save('model1')

In [110]:
model=tf.keras.models.load_model('model1')

In [133]:
x=model.predict(test_data.take(1))

1/1 [==============================] - 13s 13s/step


In [135]:
x=x[0][0]

In [136]:
def parasite(x):
  if x<0.5:
    return print('ppsitive')
  else:
    return print('Negative')  

In [137]:
parasite(x)

ppsitive
